In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/clean-data/cleandata.csv')

In [ ]:
df

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import wordnet
from random import choice, sample
from tqdm.notebook import tqdm

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
def get_wordnet_pos(treebank_tag):
    """Converts treebank tags to wordnet tags."""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    


In [ ]:
def synonym_replacement(sentence, num_replacements=2):
    """Performs synonym replacement on a sentence."""
    words = nltk.word_tokenize(sentence)
    tagged_words = nltk.pos_tag(words)
    
    replaceable_words = [word for word, tag in tagged_words if get_wordnet_pos(tag) is not None]

    replaced = 0
    for word in replaceable_words:
        synonyms = set()
        for syn in wordnet.synsets(word, pos=get_wordnet_pos(nltk.pos_tag([word])[0][1])):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())
        if word in synonyms:
            synonyms.remove(word)
        synonyms = list(synonyms)
        
        if synonyms:
            synonym = choice(synonyms).replace('_', ' ')
            sentence = sentence.replace(word, synonym, 1)
            replaced += 1
            if replaced >= num_replacements:
                break
    
    return sentence

In [ ]:
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')


In [ ]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

In [ ]:
# file_path = '/path/to/your/dataset.csv'  # Change this to your actual file path
# data = pd.read_csv(file_path)

# Prepare dataset for augmentation
class_columns = [col for col in df.columns if col != 'Text']
df['sum_classes'] = df[class_columns].sum(axis=1)
texts_with_classes = df[df['sum_classes'] > 0]

# Augment texts
num_texts_to_augment = 5  # Change as needed
augmented_texts = []

texts_to_augment = texts_with_classes.sample(n=min(num_texts_to_augment, len(texts_with_classes)))

for _, row in tqdm(texts_to_augment.iterrows(), total=texts_to_augment.shape[0], desc="Augmenting Texts"):
    original_text = row['Text']
    augmented_text = synonym_replacement(original_text, num_replacements=2)  # Adjust num_replacements as needed
    augmented_texts.append((original_text, augmented_text))

# Display original and augmented texts
for original, augmented in augmented_texts:
    print(f"Original: {original}\nAugmented: {augmented}\n")